<a href="https://colab.research.google.com/github/girija2204/DLTopics/blob/master/PdfInvoiceExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install pdfplumber

import requests
import pdfplumber
import re
from collections import namedtuple
import pandas as pd

In [70]:
Inv = namedtuple('Inv', 'vend_num vend_name inv_dt due_dt invoice_amt net_amt description')

In [5]:
def download_file(file_url):
  filename = file_url.split("/")[-1]
  with requests.get(file_url) as r:
    with open(filename, 'wb') as f:
      f.write(r.content)
  return filename

In [6]:
file_url = "https://www.tabs3.com/support/sample/apreports.pdf"
filename = download_file(file_url)

In [10]:
with pdfplumber.open(filename) as pdf:
  page = pdf.pages[16]
  text = page.extract_text()

In [13]:
print(text)

AccountsPayableReportPack
Invoice by Vendor List
Date: 08/25/2020 Invoice by Vendor List Page: 1
Jensen, Martin & Anderson, P.C.
Inv Due Inv Disc Disc Net
Voucher # Invoice # Date Date Amount Amount Date Amount 1099 Ref # Description Codes
200 United Parcel Service
200.01MLJ 3243387 032620 062620 75.00 P 75.00 1 Overnight document delivery * 1
3243387 032720 062720 75.00 P 75.00 3 Postage * 1
3243387 072620 072620 75.00 P 75.00 2 Overnight document delivery * 1
Vendor Totals 225.00 0.00 225.00
202 Software Technology, LLC
312205 082020 082520 490.00 P 490.00 1 Software Maintenance Agreement Tabs3 * 1
312206 082020 082520 239.00 P 239.00 2 Software Maintenance Agreement Tabs3 General Ledger * 1
312207 082020 082520 239.00 P 239.00 3 Software Maintenance Agreement Tabs3 Accounts Payable * 1
312208 082020 082520 239.00 P 239.00 4 Software Maintenance Agreement Tabs3 Trust Accounting * 1
312209 082020 082520 800.00 P 800.00 5 Software Maintenance Agreement PracticeMaster (9 Users) * 1
3122

In [18]:
new_vendor_re = re.compile(r"^\d{3} [A-Z].*")

In [ ]:
for line in text.split('\n'):
  if new_vendor_re.match(line):
    vendor_num, *vend_name = line.split()
    vend_name = " ".join(vend_name)
  print(vend_name)

In [63]:
inv_line_re = re.compile(r"(\d{6}) (\d{6}) ([\d,]+\.\d{2}) [\sP]*([\d,]+\.\d{2}) [YN ]*\d (.*?) [*\s\d]")

In [71]:
line_items = []
for line in text.split("\n"):
  if new_vendor_re.match(line):
    vend_num, *vend_name = line.split()
    vend_name = " ".join(vend_name)
  line = inv_line_re.search(line)
  if line:
    inv_dt = line.group(1)
    due_dt = line.group(2)
    inv_amt = line.group(3)
    net_amt = line.group(4)
    description = line.group(5)
    line_items.append(Inv(vend_num, vend_name, inv_dt, due_dt, inv_amt, net_amt, description))

In [72]:
line_items[2]

Inv(vend_num='200', vend_name='United Parcel Service', inv_dt='072620', due_dt='072620', invoice_amt='75.00', net_amt='75.00', description='Overnight document delivery')

In [76]:
df = pd.DataFrame(line_items)
df.head()

,vend_num,vend_name,inv_dt,due_dt,invoice_amt,net_amt,description
0,200,United Parcel Service,032620,062620,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,032720,062720,75.00,75.00,Postage
2,200,United Parcel Service,072620,072620,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",082020,082520,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",082020,082520,239.00,239.00,Software Maintenance Agreement Tabs3 General L...


In [77]:
df['inv_dt'] = pd.to_datetime(df['inv_dt'])
df['due_dt'] = pd.to_datetime(df['due_dt'])

In [78]:
df.head()

,vend_num,vend_name,inv_dt,due_dt,invoice_amt,net_amt,description
0,200,United Parcel Service,2020-03-26,2020-06-26,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,2020-03-27,2020-06-27,75.00,75.00,Postage
2,200,United Parcel Service,2020-07-26,2020-07-26,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",2020-08-20,2020-08-25,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",2020-08-20,2020-08-25,239.00,239.00,Software Maintenance Agreement Tabs3 General L...


In [82]:
df['invoice_amt'] = df['invoice_amt'].map(lambda x: float(x.replace(",", "")))

In [83]:
df['net_amt'] = df['net_amt'].map(lambda x: float(x.replace(",", "")))

In [86]:
df['invoice_amt'].sum()

22476.309999999998

In [88]:
df.to_csv('invoice_lines.csv')